# Rift Valley Fever ABM Model

This is the ABM model to predict an outbreak in Uganda and to test the effect of interventions: vaccination vs quarantine.

## Initialize the RVF Model

Set parameters for RVF

In [3]:
import rvf
import sciris as sc
import starsim as ss
import numpy as np


rvf_disease = rvf.rvf()

n_agents = 1000
prop_calf =  0.022

pars = sc.objdict(
    birth_rate = 32.6, #National Animal Census of 2021
    death_rate = 30, # National Animal Census of 2021
    n_agents = n_agents,
    # age = np.array(['Calf' if np.random.rand() < prop_calf else 'Adult' for _ in range(n_agents)]),
    networks = 'random',
)

sim = ss.Sim(pars=pars, diseases=rvf_disease)
sim.run()
sim.plot()

KeyNotFoundError: Key(s) ['age'] not found; available keys are ['n_agents', 'total_pop', 'pop_scale', 'remove_dead', 'location', 'birth_rate', 'death_rate', 'start', 'end', 'n_years', 'burnin', 'dt', 'dt_demog', 'rand_seed', 'slot_scale', 'verbose', 'demographics', 'diseases', 'networks', 'connectors', 'interventions', 'analyzers']

## Interventions

In [4]:
import numpy as np # Math
import sciris as sc # Utilities
import pylab as pl # Plotting
import starsim as ss # ABM


class vaccination(ss.Intervention):  # Create a new, generic treatment intervention

    def __init__(self, prob=0.12, mean_dur_infection=9):
        super().__init__() # Initialize the intervention
        self.prob = prob # Store the probability of vaccination
        self.dur_infection = ss.normal(mean_dur_infection, 1)  # Store the duration of infection

    def apply(self, sim):
        rvf = sim.diseases.rvf

        # Define  who is eligible for vaccination
        eligible_ids = sim.people.uid[rvf.ti_infected == sim.ti]  # People are eligible for treatment if they have just started exhibiting symptoms
        n_eligible = len(eligible_ids) # Number of people who are eligible

        # Define who receives  vaccination
        is_vaccinated = np.random.rand(n_eligible) < self.prob  # Define which of the n_eligible people get treated by comparing np.random.rand() to self.p
        vaccinated_ids = eligible_ids[is_vaccinated]  # Pull out the IDs for the people receiving the treatment

        # vaccinated people will have a new disease progression
        dur_inf = self.dur_infection.rvs(vaccinated_ids)  # Sample duration of infection by generating random variables (RVS) drawn from the distribution of durations

        # Determine who dies and who recovers and when
        will_die = rvf.pars.p_death.rvs(treat_ids)  # Sample probability of dying
        dead_uids = vaccinated_ids[will_die]  # Pull out the IDs of people who die
       # recover_uids = vaccinated_ids[~will_die]  # Pull out the IDs of people who recovery
        rvf.ti_dead[dead_uids] = rvf.ti_infected[dead_uids] + dur_inf[will_die] / sim.dt  # Set the time of death
        rvf.ti_recovered[recover_uids] = rvf.ti_infected[recover_uids] + dur_inf[~will_die] / sim.dt  # Set the time of recovery


class Quarantine(ss.Intervention):
    def __init__(self, prob=1, rel_trans=0, delay=0):
        super().__init__() # Initialize the intervention
        self.prob = prob # Store the probability of quarantining
        self.rel_trans = rel_trans  # Relative transmissibility of people in quarantine
        self.delay = delay  # Delay between exposure and being put into quarantine

    def apply(self, sim):
        rvf = sim.diseases.rvf

        # Define  who is eligible for vaccination
        eligible_ids = sim.people.uid[(rvf.ti_exposed - self.delay) == sim.ti]  # People are eligible for quarantine after a delay 
        n_eligible = len(eligible_ids)  # Number of eligible people

        # Define who is quarantined
        in_quarantine = np.random.rand(n_eligible) < self.prob  # Define which of the n_eligible people are quarantined by comparing np.random.rand() to self.p
        quar_ids = eligible_ids[in_quarantine]  # Pull out the IDs for the people going into quarantine

        # Quarantined people will have a reduced relative transmissibility
        rvf.rel_trans[quar_ids] = self.rel_trans


def make_run_sim(delay=0, n_contacts=7.41, beta=0.33, treated_dur_infection=8, do_plot=True, verbose=0.1):

    # Make the interventions
    vaccination = vaccination(mean_dur_infection=vaccinated_dur_infection)  # Create the specific treatment
    quarantine = Quarantine(delay=delay)  # Create the specific quarantine

    # Make the parameters
    pars = dict(start=0, end=30, dt=1.0, verbose=verbose)  # Define the parameters of the simulation
    n_contacts = n_contacts  # Define the daily number of contacts that a person has
    beta = beta  # Define the probability of an infected person transmitting to a susceptible person

    # In tutorial 5.2, the 'pars' dictionary also oncluded the diseases and the network. It looked like this:
    #    pars = dict(start=0, end=50, dt=1.0, diseases='sis', networks='random')
    # In this tutorial, we will see how to change some of the parameters of the disease and network
    # Rather than putting them in the pars dictionary, we provide them separately to the simulation. 

    # Define "baseline" and "intervention" sims: without any interventions, with treatment, and with quarantine
    baseline = ss.Sim(
        pars, 
        networks=ss.RandomNet(pars=dict(n_contacts=ss.poisson(n_contacts))),
        diseases = ss.rvf(pars=dict(beta=beta))
        )
    vaccinated_sim = ss.Sim(
        pars, 
        networks=ss.RandomNet(pars=dict(n_contacts=ss.poisson(n_contacts))),
        diseases = ss.rvf(pars=dict(beta=beta)),
        interventions=vaccinated
        )
    quar_sim = ss.Sim(
        pars,
        networks=ss.RandomNet(pars=dict(n_contacts=ss.poisson(n_contacts))),
        diseases = ss.rvf(pars=dict(beta=beta)),
        interventions=quarantine
        )

    # Run the three simulations
    baseline.run()
    vaccinated_sim.run()
    quar_sim.run()

    # Pull out results to plot
    tvec = baseline.yearvec
    n_inf_base = baseline.results.rvf.n_infected
    n_inf_vaccinated = vaccinated_sim.results.rvf.n_infected
    n_inf_quar = quar_sim.results.rvf.n_infected

    new_inf_base = baseline.results.rvf.new_infections
    new_inf_vaccinated = vaccinated_sim.results.rvf.new_infections
    new_inf_quar = quar_sim.results.rvf.new_infections

    if do_plot:  # Whether or not to make the plot
        # Create the figure
        pl.figure()

        # Plot number of people infected
        pl.subplot(2,1,1)  # Create the first panel in the plot
        pl.title('Number of people infected')
        pl.plot(tvec, n_inf_base, 'o-', label='Baseline')  # Plot baseline
        pl.plot(tvec, n_inf_vaccinated, 'o-', label='vaccination')  # Plot treatment
        pl.plot(tvec, n_inf_quar, 'o-', label='Quarantine')  # Plot quarantine
        pl.legend()

        # Plot number of new infections
        pl.subplot(2,1,2) # Create the second panel in the plot
        pl.title('New infections')
        pl.plot(tvec, new_inf_base, 'o-', label='Baseline') # Plot baseline
        pl.plot(tvec, new_inf_vaccinated, 'o-', label='vaccination') # Plot treatment
        pl.plot(tvec, new_inf_quar, 'o-', label='Quarantine') # Plot quarantine
        pl.legend()

        sc.figlayout()
        pl.show()

    # Calculate and display the number of infections averted
    averted_by_treatment = n_inf_base.sum() - n_inf_treat.sum()
    averted_by_quarantine = n_inf_base.sum() - n_inf_quar.sum()
    print(f'Number of infections averted by vaccination: {averted_by_vaccination}')
    print(f'Number of infections averted by quarantine: {averted_by_quarantine}')
    print(f'Percentage of infections averted by vaccination: {(averted_by_vaccination)/n_inf_base.sum()}')
    print(f'Percentage of infections averted by quarantine: {(averted_by_quarantine)/n_inf_base.sum()}')


# Make, run, and plot the simulation
make_run_sim()


UnboundLocalError: cannot access local variable 'vaccination' where it is not associated with a value